#Connect to sharepoint

In [1]:
!pip install office365
!pip install Office365-REST-Python-Client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.4/185.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.client_credential import ClientCredential
from office365.sharepoint.files.file import File

####inputs########
# This will be the URL that points to your sharepoint site.
# Make sure you change only the parts of the link that start with "Your"
url_shrpt = 'https://ihuedu.sharepoint.com/sites/EDYTEProject2023/'
username_shrpt = '###############'
password_shrpt = '###############'
folder_files_url_shrpt = '/sites/EDYTEProject2023/Shared%20Documents/General/wp5_data/analysis_exports/entities_extraction/entities_extracted/'
folder_analysis_url_shrpt = '/sites/EDYTEProject2023/Shared%20Documents/General/wp5_data/analysis_exports/'

In [3]:
###Authentication###For authenticating into your sharepoint site###
ctx_auth = AuthenticationContext(url_shrpt)
if ctx_auth.acquire_token_for_user(username_shrpt, password_shrpt):
  ctx = ClientContext(url_shrpt, ctx_auth)
  web = ctx.web
  ctx.load(web)
  ctx.execute_query()
  print('Authenticated into sharepoint as: ',web.properties['Title'])

else:
  print(ctx_auth.get_last_error())
############################

Authenticated into sharepoint as:  EDYTE Project 2023


In [4]:
####Function for extracting the file names of a folder in sharepoint###
###If you want to extract the folder names instead of file names, you have to change "sub_folders = folder.files" to "sub_folders = folder.folders" in the below function

global print_folder_contents
def print_folder_contents(ctx, folder_url):
    try:

        folder = ctx.web.get_folder_by_server_relative_url(folder_url)
        fold_names = []
        sub_folders = folder.files #Replace files with folders for getting list of folders
        ctx.load(sub_folders)
        ctx.execute_query()

        for s_folder in sub_folders:

            fold_names.append(s_folder.properties["Name"])

        return fold_names

    except Exception as e:
        print('Problem printing out library contents: ', e)
######################################################

# Call the function by giving your folder URL as input
filelist_shrpt=print_folder_contents(ctx,folder_files_url_shrpt)
#Print the list of files present in the folder
print(filelist_shrpt)

['process_description.csv', 'process_steps.csv', 'process_remarks.csv', 'process_steps_digital.csv', 'process_evidences.csv', 'process_conditions.csv']


# Import Data

In [5]:
import pandas as pd
import csv
import numpy as np
import io
import os
import tempfile

In [6]:
def process_csv_file(file_name):

    file_url= folder_files_url_shrpt+file_name

    response = File.open_binary(ctx, file_url)  # Assuming File is imported and ctx is defined

    df = pd.read_csv(io.BytesIO(response.content))

    column_names_list = df.columns.tolist()


    if column_names_list[0] == "id":
      df ['ihu_unique_step_id'] = 'N/A'
      df ['ihu_unique_step_digital_id'] = 'N/A'
      df ['ihu_unique_evidence_id'] = 'N/A'
      df ['ihu_unique_condition_id'] = 'N/A'

    elif column_names_list[0] == "ihu_unique_step_id":
      df ['id'] = 'N/A'
      df ['ihu_unique_step_digital_id'] = 'N/A'
      df ['ihu_unique_evidence_id'] = 'N/A'
      df ['ihu_unique_condition_id'] = 'N/A'

    elif column_names_list[0] == "ihu_unique_step_digital_id":
      df ['id'] = 'N/A'
      df ['ihu_unique_step_id'] = 'N/A'
      df ['ihu_unique_evidence_id'] = 'N/A'
      df ['ihu_unique_condition_id'] = 'N/A'

    elif column_names_list[0] == "ihu_unique_evidence_id":
      df ['id'] = 'N/A'
      df ['ihu_unique_step_id'] = 'N/A'
      df ['ihu_unique_step_digital_id'] = 'N/A'
      df ['ihu_unique_condition_id'] = 'N/A'

    elif column_names_list[0] == "ihu_unique_condition_id":
      df ['id'] = 'N/A'
      df ['ihu_unique_step_id'] = 'N/A'
      df ['ihu_unique_step_digital_id'] = 'N/A'
      df ['ihu_unique_evidence_id'] = 'N/A'

    return df

In [7]:
#steps
entities_process_steps = process_csv_file("process_steps.csv")

#steps digital
entities_process_steps_digital = process_csv_file("process_steps_digital.csv")

#process description
entities_process_description = process_csv_file("process_description.csv")

#process remarks
entities_process_remarks = process_csv_file("process_remarks.csv")

#process evidences
entities_process_evidences = process_csv_file("process_evidences.csv")

#process conditions
entities_process_conditions = process_csv_file("process_conditions.csv")

## Combine

In [100]:
# Concatenate the data frames
combined = pd.concat([entities_process_description, entities_process_remarks, entities_process_steps, entities_process_steps_digital, entities_process_evidences,entities_process_conditions], ignore_index=True, sort=False)

# Reorder the columns
combined = combined[['id', 'ihu_unique_step_id', 'ihu_unique_step_digital_id', 'ihu_unique_evidence_id', 'ihu_unique_condition_id', 'word']]

In [101]:
combined = combined.drop_duplicates()

In [102]:
combined

,id,ihu_unique_step_id,ihu_unique_step_digital_id,ihu_unique_evidence_id,ihu_unique_condition_id,word
0,523764,N/A,N/A,N/A,N/A,ελγα
1,523764,N/A,N/A,N/A,N/A,οργανισμου ελληνικων γεωργικων ασφαλισεων
2,820937,N/A,N/A,N/A,N/A,e εφκα
3,963466,N/A,N/A,N/A,N/A,ιδιωτικων επιχειρησεων παροχης υπηρεσιων ασφαλ...
4,963466,N/A,N/A,N/A,N/A,"κεντρων ληψης, ελεγχου και διαβιβασης σηματων ..."
...,...,...,...,...,...,...
63224,N/A,N/A,N/A,N/A,135964_con_01,φορτηγο δημοσιας
63225,N/A,N/A,N/A,N/A,135964_con_06,ιδιωτικης
63226,N/A,N/A,N/A,N/A,373136_con_01,φορτηγο
63227,N/A,N/A,N/A,N/A,373136_con_01,αυτοκινητου


In [103]:
mask = (combined['word'].str.len() <= 2) | (combined['word'].str.contains('/')) | (combined['word'].str.contains('[A-Za-z]'))

# Drop the rows that meet the conditions
combined = combined[~mask]

In [104]:
combined

,id,ihu_unique_step_id,ihu_unique_step_digital_id,ihu_unique_evidence_id,ihu_unique_condition_id,word
0,523764,N/A,N/A,N/A,N/A,ελγα
1,523764,N/A,N/A,N/A,N/A,οργανισμου ελληνικων γεωργικων ασφαλισεων
3,963466,N/A,N/A,N/A,N/A,ιδιωτικων επιχειρησεων παροχης υπηρεσιων ασφαλ...
4,963466,N/A,N/A,N/A,N/A,"κεντρων ληψης, ελεγχου και διαβιβασης σηματων ..."
5,439605,N/A,N/A,N/A,N/A,οργανισμου ελληνικων γεωργικων ασφαλισεων
...,...,...,...,...,...,...
63224,N/A,N/A,N/A,N/A,135964_con_01,φορτηγο δημοσιας
63225,N/A,N/A,N/A,N/A,135964_con_06,ιδιωτικης
63226,N/A,N/A,N/A,N/A,373136_con_01,φορτηγο
63227,N/A,N/A,N/A,N/A,373136_con_01,αυτοκινητου


In [105]:
docs = combined["word"].values.tolist()

In [106]:
len(docs)

55276

#Import Models

In [14]:
%%capture
!pip install bertopic

In [15]:
import pandas as pd

# Topic Modeling

## Stop Words

In [16]:
# get Greek stop_words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
greek_stopwords = stopwords.words('greek')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
new_words = ['της', 'τη', 'του', 'από']

for word in new_words:
  greek_stopwords.append(word)

## Initialise Models

In [18]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer # Embeddings
from umap import UMAP #Dimensionality reduction
from hdbscan import HDBSCAN #clustering
from sklearn.feature_extraction.text import CountVectorizer # CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from transformers.pipelines import pipeline

In [19]:
embedding_model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")
umap_model = UMAP(n_neighbors=100, n_components=10, min_dist=0.5, metric='cosine') #increasing n_neighbors in general larger clusters | n_components - in which dimension. Too small loss of info, too high performance | min_dist how far the points should be in low dimensional
hdbscan_model = HDBSCAN(min_cluster_size=25, metric='euclidean', cluster_selection_method='eom') #min_cluster_size, increasing -> fewer clusters | cluster_selection_method also the "leaf" method available (smaller clusters)
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=greek_stopwords) #ngram_range combination of words
ctfidf_model = ClassTfidfTransformer(bm25_weighting=True)

In [20]:
topic_model = BERTopic(embedding_model=embedding_model, umap_model=umap_model, hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model, ctfidf_model=ctfidf_model)
topics, probs = topic_model.fit_transform(docs)

## Extracting Topics

In [21]:
freq = topic_model.get_topic_info(); freq.head(20)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12519,-1_πορων_οργανωσης_εκπαιδευτικου προσωπικου_δι...,"[πορων, οργανωσης, εκπαιδευτικου προσωπικου, δ...",[διευθυνσης διοικησης εκπαιδευτικου προσωπικου...
1,0,985,0_εκε κεπ κεπ_κεπ κεπ εκε_κτελ_κεκ,"[εκε κεπ κεπ, κεπ κεπ εκε, κτελ, κεκ, κεπ εκε ...","[κεπ, κεπ, κεπ]"
2,1,447,1____,"[, , , , , , , , , ]","[ε. ε, ε. ε, ε. ε]"
3,2,382,2_εταα_εταα εταα__,"[εταα, εταα εταα, , , , , , , , ]","[τ. ε. ε, τ. εταα, τ. εταα]"
4,3,307,3_πρωτοδικων_εισαγγελεα_δικαστικου_δικαστηριου,"[πρωτοδικων, εισαγγελεα, δικαστικου, δικαστηρι...","[εισαγγελεα πρωτοδικων, εισαγγελεα πρωτοδικων,..."
5,4,280,4_ευρωπαικης ενωσης ευρωπαικης_ενωσης ευρωπαικ...,"[ευρωπαικης ενωσης ευρωπαικης, ενωσης ευρωπαικ...","[ευρωπαικης ενωσης, ευρωπαικης ενωσης, ευρωπαι..."
6,5,272,5_λιμενικη αρχη_λιμενικη_αρχη λιμενικη_λιμενικ...,"[λιμενικη αρχη, λιμενικη, αρχη λιμενικη, λιμεν...","[λιμενικη αρχη, λιμενικη αρχη, λιμενικη αρχη]"
7,6,258,6_γραφειου πρωτοκολλου_πρωτοκολλου γραφειου πρ...,"[γραφειου πρωτοκολλου, πρωτοκολλου γραφειου πρ...","[γραφειου πρωτοκολλου, γραφειου πρωτοκολλου, γ..."
8,7,231,7_ελληνικης αστυνομιας ελληνικης_αστυνομιας ελ...,"[ελληνικης αστυνομιας ελληνικης, αστυνομιας ελ...","[ελληνικης αστυνομιας, ελληνικης αστυνομιας, ε..."
9,8,230,8_διευθυντη διευθυντη_διευθυντη διευθυντη διευ...,"[διευθυντη διευθυντη, διευθυντη διευθυντη διευ...","[διευθυντης, διευθυντης, διευθυντης]"


In [22]:
freq["Topic"].nunique()

607

For -1

In [23]:
new_topics = topic_model.reduce_outliers(docs, topics, strategy="embeddings", threshold=0.70)

In [24]:
topic_model.update_topics(docs, topics=new_topics, vectorizer_model=vectorizer_model,  ctfidf_model=ctfidf_model)

In [25]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,722,-1_προωθησης αδειοδοτησης τουριστικων_τουριστι...,"[προωθησης αδειοδοτησης τουριστικων, τουριστικ...",[διευθυνσης διοικησης εκπαιδευτικου προσωπικου...
1,0,987,0_εκε κεπ κεπ_κεπ κεπ εκε_κτελ_κεκ,"[εκε κεπ κεπ, κεπ κεπ εκε, κτελ, κεκ, κεπ εκε ...","[κεπ, κεπ, κεπ]"
2,1,449,1____,"[, , , , , , , , , ]","[ε. ε, ε. ε, ε. ε]"
3,2,382,2_εταα_εταα εταα__,"[εταα, εταα εταα, , , , , , , , ]","[τ. ε. ε, τ. εταα, τ. εταα]"
4,3,385,3_εισαγγελεα_δικαστικο_πρωτοδικων_δικαστικων,"[εισαγγελεα, δικαστικο, πρωτοδικων, δικαστικων...","[εισαγγελεα πρωτοδικων, εισαγγελεα πρωτοδικων,..."


# Save Topic

In [109]:
def upload_csv_to_target_folder(name, main_df):

  path = name+".csv"

  result = topic_model.get_document_info(docs)

  main_df = main_df.reset_index()
  main_df ["topic"] = result['Topic']
  main_df.to_csv(path, index=False)

  url=folder_analysis_url_shrpt+"entities_extraction/"

  target_folder = ctx.web.get_folder_by_server_relative_url(url)
  with open(path, "rb") as content_file:
      file_content = content_file.read()
      target_folder.upload_file(os.path.basename(path), file_content).execute_query()

In [110]:
upload_csv_to_target_folder('entities_topic', combined)